**I executed  SQL queries against the 3 tables below.**
---
Table 'dataset' has ids of films and directors from tables 'films' and 'directors'

In [ ]:
import pandas as pd
dataset = pd.read_csv('dataset.csv')
films = pd.read_csv('films.csv')

directors = pd.read_csv('directorsrecto_columns.to_list()) = films_columns = films.columns.to_list()
directorsrs.columnsprint('The The  table "films" has columns:', films_columns).to_list()rs.csv')

The table "dataset" has columns: ['index', 'film_id', 'year', 'runtime', 'genre', 'rating', 'director_id']
The table "films" has columns: ['film_id', 'film_title']
The table "directors" has columns: ['director_id', 'director']


# SQL queries (various)

* Top 5 directors with the highest average rating

In [ ]:
%%sql @noteable
select director, round(avg(rating),2) as average_rating from dataset
left join directors using(director_id)
group by director
order by average_rating desc
limit 5;

* Films directed by Akira Kurosawa or 
Martin Scorsese and released between 1980 and 1990

In [ ]:
%%sql @noteable
select film_title, director, year from dataset
left join films using(film_id)
left join directors using(director_id)
where year between 1980 and 1990 and director in ('Akira Kurosawa','Martin Scorsese');

* Unique directors

In [ ]:
%%sql @noteable
select distinct director from directors;

* Minimum, maximum and average rating of all films in the dataset

In [ ]:
%%sql @noteable
select min(rating) as minimum_rating, max(rating) as maximum_rating, round(avg(rating),1) as average_rating from dataset;

* Minimum and maximum rating for each director

In [ ]:
%%sql @noteable
select director, min(rating) as minimum_rating, max(rating) as maximum_rating from
dataset left join directors using(director_id)
group by director
order by minimum_rating desc;

* 3 random films

In [ ]:
%%sql @noteable
select film_title from films
order by random()
limit 3;

* Film description including film title, year and director

In [ ]:
%%sql @noteable
select film_title, concat('"',film_title,'"',' ','was released in',' ', year,' ','and',' ','directed by',
              ' ', director) as film_description 
from dataset 
left join films using(film_id)
left join directors using(director_id);

* Films that have a rating which is higher than the average rating of all the films in the dataset

# Subqueries

In [ ]:
%%sql @noteable
select film_title, rating, (select round(avg(rating),2) from dataset) as average_rating from dataset inner join films using (film_id)
where rating > average_rating;

* Number of unique directors
* Number of films
* Average number of films per director
* Maximum number of films per director
* Minimum number of films per director

In [ ]:
%%sql @noteable
select 
count(distinct(director_id)) as num_of_dir, 
count(film_id) as num_of_films, 
count(film_id)/count(distinct(director_id)) as avg_films_per_dir,
(select max(counts) from (select count(film_id) as counts from dataset
 group by director_id)) as max_film_per_dir,
 
 (select min(counts) from (select count(film_id) as counts from dataset
 group by director_id)) as min_film_per_dir

from dataset;

* The titles of the drama films with the longest runtime

In [ ]:
%%sql @noteable
select film_title, runtime, genre from dataset
left join films using(film_id)
where genre like '%Drama%' and runtime in (select max(runtime) from dataset where genre like '%Drama%');

# CTE, IF, CASE WHEN

* Split films into groups using the length. Find the numberf films in each group.

In [ ]:
%%sql @noteable
with t1 (film_title, runtime,length) as
(select film_title, runtime, if(runtime > 120, 'long', if(runtime<60, 'short', 'average')) as length from dataset
left join films using (film_id))

select count(film_title) as amount_of_films, length from t1
group by length

;

* Create a column defining a century in which a film was released

In [ ]:
%%sql @noteable
select film_title, year, case
    when year>1999 THEN '21 century'
    else '20 century'
end as century
from dataset left join films using(film_id);

* Directors who directed 5 and more films released  21st century

In [ ]:
%%sql @noteable
with t1 (film_title, year, runtime, genre, rating, director) as
(select film_title, year, runtime, genre, rating, director from dataset 
 left join films using(film_id)
 left join directors using(director_id)
 where year >= 2000)
 
 select director, count(film_title) as num_of_films from t1
 group by director
 having count(film_title) >=5
 order by num_of_films desc
 ;

# Window Functions

* Average films runtime per director

In [ ]:
%%sql @noteable
select distinct director,
       round(avg(runtime) OVER (partition by director)) AS avg_time_per_dir
from dataset left join directors using(director_id)
order by avg_time_per_dir ;

* Number the films for every director by year

In [ ]:
%%sql @noteable
select director, film_title, 
row_number() over (partition by director order by year) as rank,
year
from dataset left join directors using(director_id) left join films using(film_id);

* Rank films by rating

In [ ]:
%%sql @noteable
select film_title, rating, dense_rank() over (order by rating desc) as rank 
from dataset left join films using(film_id);

* Calculate the difference between the last and the first film of each director in the dataset. Order by descending difference.

In [ ]:
%%sql @noteable
with t1 (director, year, rank) as 
(select director, year, 
row_number() over (partition by director_id order by year) as rank
from dataset left join directors using(director_id))
select distinct director, (last_value(year) OVER (partition by director)) - (first_value(year) OVER (partition by director)) as difference
from t1
order by difference desc
;

# UNION, INTERSECT, EXCEPT

* Select information about the films which were released in 2005 and in 2011

In [ ]:
%%sql @noteable
select * from dataset
where year = 2011
union all
select * from dataset
where year = 2005
;

* Select information about the films which were released after 2011 and after 2008 (without duplicates)

In [ ]:
%%sql @noteable
select * from dataset
where year > 2011
union
select * from dataset
where year > 2008
order by film_id;

* Select information about the films which were released before 2011 but after 2008

In [ ]:
%%sql @noteable
select * from dataset
where year < 2011
intersect
select * from dataset
where year > 2008


* Select information about the films which were released after 2006 except films released in 2008

In [ ]:
%%sql @noteable
select * from dataset
where year > 2006
except
select * from dataset
where year = 2008
order by year;